<a href="https://colab.research.google.com/github/SamuelSousaFerreira/Mercado-Financeiro/blob/main/Case_MZR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Carregando os dados**

In [ ]:
import pandas as pd #operações com tabelas
import numpy as np #operações matemáticas
!pip install xlsxwriter #gerar o excel com várias abas de clientes
import calendar # operações com data
import datetime #operações com data

     |████████████████████████████████| 153kB 2.9MB/s 


In [ ]:
arquivo_excel = pd.ExcelFile("/content/drive/MyDrive/Portfólio/Case da MZR/Desafio VBA.xlsx") #Carrego o excel

operações = pd.read_excel(arquivo_excel, sheet_name='Trade Output', header=1)# index_col= 'Date & Time' # coloco em um DataFrame as abas desejadas
desafio = operações.copy()
desafio.head()

,Unnamed: 0,Date & Time,Stock,B/S,Amount,Client,Price
0,NaN,2021-05-07 13:32:31.350,BBDC3,B,5,1040,20.82
1,NaN,2021-05-07 13:32:32.214,PETR4,S,5,1040,25.19
2,NaN,2021-05-07 13:32:33.078,SANB11,S,20,Mesa,38.91
3,NaN,2021-05-07 13:32:33.942,PETR4,S,10,2806,23.54
4,NaN,2021-05-07 13:32:34.806,BBDC3,B,10,2030,21.11


In [ ]:
desafio.drop(columns='Unnamed: 0', inplace=True) #excluindo a coluna em branco
desafio.head() #conferindo

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,B,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,S,5,1040,25.19
2,2021-05-07 13:32:33.078,SANB11,S,20,Mesa,38.91
3,2021-05-07 13:32:33.942,PETR4,S,10,2806,23.54
4,2021-05-07 13:32:34.806,BBDC3,B,10,2030,21.11


In [ ]:
desafio.info()
#Verificando os dados e a tipagem
#não há dados faltantes e nem nulos nas 1000 linhas
#tipagem feita corretamente, portanto não é necessário alterar nada.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date & Time  1000 non-null   datetime64[ns]
 1   Stock        1000 non-null   object        
 2   B/S          1000 non-null   object        
 3   Amount       1000 non-null   int64         
 4   Client       1000 non-null   object        
 5   Price        1000 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 47.0+ KB


In [ ]:
#A coluna B/S, significa B=Buy e S=Sell, então substituiremos B por 1 e S por -1.
compra_venda = {'B':1,'S':-1}
desafio['B/S'] = desafio['B/S'].map(compra_venda)
#esafio['Compra/Venda'] = desafio['B/S'].map(compra_venda) ou criar uma nova coluna
desafio.head()

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,1,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,-1,5,1040,25.19
2,2021-05-07 13:32:33.078,SANB11,-1,20,Mesa,38.91
3,2021-05-07 13:32:33.942,PETR4,-1,10,2806,23.54
4,2021-05-07 13:32:34.806,BBDC3,1,10,2030,21.11


In [ ]:
#Regra:Posições vendidas devem entrar com sinal negativo nas carteiras.
#Tanto as quantidades quanto o preço receberão o sinal negativo quando a operaçõe for de venda
desafio['Amount'] = desafio['B/S'] * desafio.Amount
desafio['Price'] = desafio['B/S'] * desafio.Price
desafio.head()

,Date & Time,Stock,B/S,Amount,Client,Price
0,2021-05-07 13:32:31.350,BBDC3,1,5,1040,20.82
1,2021-05-07 13:32:32.214,PETR4,-1,-5,1040,-25.19
2,2021-05-07 13:32:33.078,SANB11,-1,-20,Mesa,-38.91
3,2021-05-07 13:32:33.942,PETR4,-1,-10,2806,-23.54
4,2021-05-07 13:32:34.806,BBDC3,1,10,2030,21.11


In [ ]:
# Criando uma lista de Clientes da base para posterior agrupamento
clientes = list(desafio.Client.unique()) 
clientes

[1040, 'Mesa', 2806, 2030, 1917, 1150, 3018]

# **Mesa**
As boletas registradas sob o operador “Mesa” devem ser especificadas para TODOS os clientes de forma equânime, ou seja, com o mesmo preço médio de forma que não haja seleção de preço por cliente (Fair Dealing).  

In [ ]:
#filtrando as operações da Mesa
mesa = desafio.loc[desafio.Client == 'Mesa']
mesa.head()

,Date & Time,Stock,B/S,Amount,Client,Price
2,2021-05-07 13:32:33.078,SANB11,-1,-20,Mesa,-38.91
8,2021-05-07 13:32:38.262,ITUB4,1,15,Mesa,29.43
41,2021-05-07 13:33:06.774,BBDC3,1,5,Mesa,22.97
43,2021-05-07 13:33:08.502,BPAC11,1,5,Mesa,112.02
49,2021-05-07 13:33:13.686,MGLU3,1,15,Mesa,20.80


## Compra Mesa

In [ ]:
#Quantidade de Ações compradas
mesa_compr = mesa.loc[mesa['B/S'] == 1]
mesa_compr.head()

,Date & Time,Stock,B/S,Amount,Client,Price
8,2021-05-07 13:32:38.262,ITUB4,1,15,Mesa,29.43
41,2021-05-07 13:33:06.774,BBDC3,1,5,Mesa,22.97
43,2021-05-07 13:33:08.502,BPAC11,1,5,Mesa,112.02
49,2021-05-07 13:33:13.686,MGLU3,1,15,Mesa,20.80
52,2021-05-07 13:33:16.278,ITUB4,1,10,Mesa,28.68


In [ ]:
#lista de ações mesa
stock_mesa = list(mesa.Stock.unique())
stock_mesa

['SANB11', 'ITUB4', 'BBDC3', 'BPAC11', 'MGLU3', 'PETR4']

In [ ]:
#Criando um Data Frame para armazenar os Resultados
neg_mesa = pd.DataFrame(columns=['Stock','Amount','Price'])
neg_mesa

,Stock,Amount,Price


In [ ]:
#Calculando a média por ação e armazeando no DF
for x in stock_mesa:
  acoes_x = mesa_compr.loc[mesa_compr.Stock == x]
  negociadas = acoes_x.Amount.sum()
  preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2) #media ponderada arredondando para 2 casas decimais
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)


In [ ]:
neg_mesa

,Stock,Amount,Price
0,SANB11,170,39.91
1,ITUB4,150,27.99
2,BBDC3,130,21.34
3,BPAC11,195,111.75
4,MGLU3,205,20.20
5,PETR4,220,24.93


## Venda Mesa

In [ ]:
#Ações Vendidas
mesa_vend = mesa.loc[mesa['B/S'] == -1]
mesa_vend.head()

for x in stock_mesa:
  acoes_x = mesa_vend.loc[mesa_vend.Stock == x]
  negociadas = acoes_x.Amount.sum()
  preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

#Criando uma planilha xlsx para armazenar nas abas as posições de cada cliente
operacoes_mesa = pd.ExcelWriter('operacoes_mesa.xlsx', engine='xlsxwriter') 

#adicionando as operações em abas do xlsx
neg_mesa.to_excel(operacoes_mesa, sheet_name= 'Operações Mesa') 

## Resultado da Carteira Mesa

In [ ]:
#tirar o sinal negativo de price e nao alterar o registro principal
neg_mesa_1 =neg_mesa.copy()
neg_mesa_1['Price'] = abs(neg_mesa_1['Price'])
neg_mesa_1

,Stock,Amount,Price
0,SANB11,170,39.91
1,ITUB4,150,27.99
2,BBDC3,130,21.34
3,BPAC11,195,111.75
4,MGLU3,205,20.20
5,PETR4,220,24.93
6,SANB11,-185,39.71
7,ITUB4,-125,28.36
8,BBDC3,-75,20.51
9,BPAC11,-190,111.97


In [ ]:
#agrupando por ação e calculando a média. Considerei para a média aritmética dos preços de venda e compra.
posição_mesa = neg_mesa_1.groupby('Stock').agg({'Amount':'sum',
                               'Price': 'mean'})

posição_mesa.to_excel(operacoes_mesa, sheet_name= 'Posição Mesa')

operacoes_mesa.save()

posição_mesa

,Amount,Price
Stock,,
BBDC3,55,20.925
BPAC11,5,111.860
ITUB4,25,28.175
MGLU3,90,19.805
PETR4,-25,25.135
SANB11,-15,39.810


## Tudo Junto

In [ ]:
#filtrando as operações da Mesa
mesa = desafio.loc[desafio.Client == 'Mesa']

#lista de ações mesa
stock_mesa = list(mesa.Stock.unique())

#Criando um Data Frame para armazenar os Resultados
neg_mesa = pd.DataFrame(columns=['Stock','Amount','Price'])
neg_mesa

#Ações compradas
mesa_compr = mesa.loc[mesa['B/S'] == 1]
mesa_compr.head()

#Ações Vendidas
mesa_vend = mesa.loc[mesa['B/S'] == -1]
mesa_vend.head()

#Calculando a média por ação e armazeando no DF
for x in stock_mesa:
  acoes_x = mesa_compr.loc[mesa_compr.Stock == x]
  negociadas = acoes_x.Amount.sum()
  preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2) #media ponderada arredondando para 2 casas decimais
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)


for x in stock_mesa:
  acoes_x = mesa_vend.loc[mesa_vend.Stock == x]
  negociadas = acoes_x.Amount.sum()
  preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais
  neg_mesa = neg_mesa.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

#Criando uma planilha xlsx para armazenar nas abas as posições de cada cliente
operacoes_mesa = pd.ExcelWriter('operacoes_mesa.xlsx', engine='xlsxwriter') 

#adicionando as operações em abas do xlsx
neg_mesa.to_excel(operacoes_mesa, sheet_name= 'Operações Mesa') 

#tirar o sinal negativo de price e nao alterar o registro principal
neg_mesa_1 =neg_mesa.copy()
neg_mesa_1['Price'] = abs(neg_mesa_1['Price'])
neg_mesa_1

#agrupando por ação e calculando a média. Considerei para a média aritmética dos preços de venda e compra.
posição_mesa = neg_mesa_1.groupby('Stock').agg({'Amount':'sum',
                               'Price': 'mean'})

posição_mesa.to_excel(operacoes_mesa, sheet_name= 'Posição Mesa')

operacoes_mesa.save()

posição_mesa


,Amount,Price
Stock,,
BBDC3,55,20.925
BPAC11,5,111.860
ITUB4,25,28.175
MGLU3,90,19.805
PETR4,-25,25.135
SANB11,-15,39.810


# **Para Qualquer Cliente**

In [ ]:
clientes

[1040, 'Mesa', 2806, 2030, 1917, 1150, 3018]

In [ ]:
#Criando uma planilha xlsx para armazenar nas abas as posições de cada cliente
posicoes_clientes = pd.ExcelWriter('posicoes_clientes.xlsx', engine='xlsxwriter') 

for y in clientes:
  a = desafio.loc[desafio.Client == y] #armazena todas as posições do cliente y
  nome_cliente = 'Client - '+ str(y) 
 
  #lista de ações do cliente y
  stock_client = list(a.Stock.unique()) 

  #Criando um Data Frame vazio para armazenar os Resultados, ele é limpado a cada iteração
  neg_client = pd.DataFrame(columns=['Stock','Amount','Price'])  

  #Ações compradas do cliente y
  a_compr = a.loc[a['B/S'] == 1]  

  #Ações Vendidas do cliente y
  a_vend = a.loc[a['B/S'] == -1]
  
  #Calculando a média ponderada por ação comprada e armazeando no DataFrame
  for x in stock_client:
    acoes_x = a_compr.loc[a_compr.Stock == x]
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais
    #print(volume_x, negociadas, preco_medio)
    neg_client = neg_client.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)
     
  #Calculando a média ponderada por ação vendida e armazeando no DataFrame
  for x in stock_client:
    acoes_x = a_vend.loc[a.Stock == x]
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais  
    neg_client = neg_client.append({"Stock":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

  #posição final do cliente
  neg_client['Price'] = abs(neg_client['Price']) #deixando todos os valores positivos para calcular a média
  neg_client = neg_client.groupby('Stock').agg({'Amount':'sum','Price':'mean'})
  
  #Regra para eliminar, caso tenha, linhas com posições igual a zero
  neg_client.drop(list(neg_client[neg_client.Amount == 0].index), inplace=True)

  #organizando em ordem alfabética
  neg_client.sort_values(by='Stock', ascending=True, inplace=True) 

  # colocando a coluna Stock como índice
  #neg_client.set_index('Stock', inplace=True) 

  neg_client.to_excel(posicoes_clientes, sheet_name= nome_cliente) #adiciona as operações em abas do xlsx

  display(nome_cliente, neg_client) #exibindo as posições de cada cliente no período

posicoes_clientes.save()  #salvando o xlsx 
  

'Client - 1040'

,Amount,Price
Stock,,
BBDC3,-115,20.395
BPAC11,60,111.955
ITUB4,205,27.790
MGLU3,-40,19.685
PETR4,170,25.340
SANB11,55,39.690


'Client - Mesa'

,Amount,Price
Stock,,
BBDC3,55,20.925
BPAC11,5,111.860
ITUB4,25,28.175
MGLU3,90,19.805
PETR4,-25,25.135
SANB11,-15,39.810


'Client - 2806'

...

,Amount,Price
Stock,,
BBDC3,20,20.615
BPAC11,30,111.485
ITUB4,-70,28.050
MGLU3,-145,20.610
PETR4,140,25.280
SANB11,-25,40.060


'Client - 1150'

,Amount,Price
Stock,,
BBDC3,35,21.410
BPAC11,40,112.145
ITUB4,10,28.035
MGLU3,25,20.365
PETR4,30,25.180
SANB11,185,40.560


'Client - 3018'

,Amount,Price
Stock,,
BBDC3,-35,21.225
BPAC11,5,112.065
ITUB4,95,27.800
MGLU3,-80,19.775
PETR4,-90,25.380
SANB11,20,40.085


# **Desafio Adicional**

Desafio adicional:
Uma carteira já possui posições prévias, então a macro deve ser capaz de calcular o novo preço médio considerando as posições já detidas pelo cliente.
Se o cliente por acaso zerar uma posição que tenha na carteira, exclua a linha deste ativo.


In [ ]:
#Carregando o arquivo da carteira prévia
arquivo_excel_2 = pd.ExcelFile("/content/drive/MyDrive/Portfólio/Case da MZR/Desafio VBA.xlsx") #Carrego o excel

Cliente = pd.read_excel(arquivo_excel_2, sheet_name='Client - 1150', header=3)# index_col= 'Date & Time' # coloco em um DataFrame as abas desejadas
cliente_1150 = Cliente.copy()
cliente_1150.drop(columns='Unnamed: 0', inplace= True)
cliente_1150

,Ticker,Amount,Price,Sector
0,BBDC3,20,20.82,Banking
1,PETR4,10,25.19,Oil and Gas
2,SANB11,5,38.91,Banking
3,MGLU3,5,18.48,Retail
4,ITUB4,10,28.07,Banking
5,BPAC11,5,110.50,Banking


In [ ]:
#separando os setores para juntá-los no final
# setores = dict(zip(ações,setor))
setores = {"BBDC3":"Banking",'PETR4':'Oil and Gas','SANB11':'Banking','MGLU3':'Retail','ITUB4':'Banking', 'BPAC11':'Banking' }
#setores

In [ ]:
#carregando o arquivo com todas as negociações do cliente do arquivo desafio

cliente_1150_2 = desafio[desafio.Client == 1150][['Stock','Amount','Price']] #escolho somente as 3 colunas para juntar com os dados da carteira
cliente_1150_2

,Stock,Amount,Price
11,SANB11,-5,-40.24
13,BPAC11,-10,-110.50
14,ITUB4,-5,-27.33
19,PETR4,15,24.45
24,BBDC3,-25,-22.32
...,...,...,...
971,BPAC11,-20,-112.29
974,PETR4,5,26.65
976,PETR4,-10,-23.05
984,BPAC11,15,112.08


In [ ]:
#renomeando a coluna stock para poder juntar as duas tabelas
cliente_1150_2.rename(columns={'Stock':'Ticker'}, inplace=True)

#juntando as tabelas
cliente_1150_posições = pd.concat([cliente_1150_2,cliente_1150], axis=0)

#excluindo a coluna Setor, mas não é obrigatório
cliente_1150_posições.drop(columns=['Sector'], inplace=True)
                                  
cliente_1150_posições

,Ticker,Amount,Price
11,SANB11,-5,-40.24
13,BPAC11,-10,-110.50
14,ITUB4,-5,-27.33
19,PETR4,15,24.45
24,BBDC3,-25,-22.32
...,...,...,...
1,PETR4,10,25.19
2,SANB11,5,38.91
3,MGLU3,5,18.48
4,ITUB4,10,28.07


In [ ]:
#ações presentes na carteira
stock_client = list(cliente_1150_posições.Ticker.unique())
stock_client

['SANB11', 'BPAC11', 'ITUB4', 'PETR4', 'BBDC3', 'MGLU3']

In [ ]:
neg_client = pd.DataFrame(columns=['Ticker','Amount','Price'])  
neg_client

,Ticker,Amount,Price


In [ ]:
#ações compradas
a_compr = cliente_1150_posições[cliente_1150_posições.Amount > 0]

for x in stock_client:
    acoes_x = a_compr.loc[a_compr.Ticker == x]
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais   
    neg_client = neg_client.append({"Ticker":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

#ações vendidas
a_vend = cliente_1150_posições[cliente_1150_posições.Amount < 0]

for x in stock_client:
    acoes_x = a_vend.loc[a_vend.Ticker == x]
    negociadas = acoes_x['Amount'].sum()
    preco_medio = (np.average(acoes_x.Price,weights=acoes_x.Amount)).round(2)  #media ponderada arredondando para 2 casas decimais)
    neg_client = neg_client.append({"Ticker":x,"Amount":negociadas,'Price':preco_medio}, ignore_index=True)

In [ ]:
neg_client

,Ticker,Amount,Price
0,SANB11,270,39.96
1,BPAC11,185,112.18
2,ITUB4,205,28.18
3,PETR4,245,25.28
4,BBDC3,270,21.26
5,MGLU3,150,19.77
6,SANB11,-80,-41.14
7,BPAC11,-140,-112.06
8,ITUB4,-185,-27.89
9,PETR4,-205,-25.07


In [ ]:
# Aqui considerei colocar o preço médio entre as ações vendidas e compradas para calcular o calor final
#por isso deixei todos os valores positivos
neg_client['Price'] = abs(neg_client.Price)
neg_client.sort_values(by='Ticker')

,Ticker,Amount,Price
4,BBDC3,270,21.26
10,BBDC3,-215,21.53
1,BPAC11,185,112.18
7,BPAC11,-140,112.06
2,ITUB4,205,28.18
8,ITUB4,-185,27.89
5,MGLU3,150,19.77
11,MGLU3,-120,20.91
3,PETR4,245,25.28
9,PETR4,-205,25.07


In [ ]:
# calculando a posição final e média por ação
neg_client = neg_client.groupby('Ticker').agg({'Amount':sum, 'Price': 'mean'})
neg_client

,Amount,Price
Ticker,,
BBDC3,55,21.395
BPAC11,45,112.120
ITUB4,20,28.035
MGLU3,30,20.340
PETR4,40,25.175
SANB11,190,40.550


In [ ]:
#inserindo a coluna sector
neg_client['Sector'] = neg_client.index.map(setores)
neg_client

,Amount,Price,Sector
Ticker,,,
BBDC3,55,21.395,Banking
BPAC11,45,112.120,Banking
ITUB4,20,28.035,Banking
MGLU3,30,20.340,Retail
PETR4,40,25.175,Oil and Gas
SANB11,190,40.550,Banking


In [ ]:
#Salvando em arquivo excel
#optei por salvar em um diferente mas poderia salvar no mesmo arquivo do tópico antetior 
posicoes_cliente_1150 = pd.ExcelWriter('posicoes_cliente_1150.xlsx', engine='xlsxwriter') 

neg_client.to_excel(posicoes_cliente_1150, sheet_name='Client - 1150')

posicoes_cliente_1150.save()